# Graph Experiment 2: Experiment Preparation
Set up experiments for the bots to do.

In [ ]:
%load_ext autoreload
%autoreload 2

## Capture code as strings
We collect code from chosen cells as strings, and place them in nodes in the graph. The code is intended to be sufficient to reproduce the results.

In [ ]:
code_strings = []

In [ ]:
import numpy as np
from nnbench import NetMaker, NNMEG
import secrets
import time
import tools.neotools as nj

In [ ]:
code_strings.append(In[-2]) # Grab the cell above

In [ ]:
from collections import defaultdict
import neo4j

## Connecting

In [8]:
driver = neo4j.GraphDatabase.driver("neo4j://neo4j:7687", auth=("neo4j", "test"))

In [ ]:
driver.verify_connectivity()

# The graph database

In [ ]:
from IPython.display import SVG, display
display(SVG('ml graphdb structure r2.svg'))

# Prepare code for later `eval`

## Create the net we will train

In [ ]:
mnm = NetMaker(NNMEG)
net = mnm('1x8tx8tx3tx3t')

def adc(input):
    m = max(0, min(7, int(8*input)))
    return np.array([(m>>2)&1, (m>>1)&1, m&1]) * 2 - 1

vadc = lambda v: np.array([adc(p) for p in v])

x = np.arange(0, 1, 1.0/(8*8)).reshape(-1,1) # 8 points in each output region
training_batch = (x, vadc(x))

In [ ]:
code_strings.append(In[-2])

## First net node

In [ ]:
def add_net_start_to_graph(driver, facts):
    q = """
MATCH (par:Parameters {unikey: $parameters_unikey})
CREATE (par)-[:configures]->(:net 
            {shorthand: $shorthand,
                   ksv: $ksv,
                  loss: $loss,
                    ts: timestamp(),
                  head: $head,
    batches_from_start: 0})
"""

    nj.query_write(driver, q, **facts)

In [ ]:
code_strings.append(In[-2])

## Trained net nodes

In [ ]:
def add_net_subsequent_to_graph(driver, facts):
    q = """
MATCH (a:net {ksv: $prior_ksv})
MERGE (a)-[:LEARNED
         {batch_points: $batch_points,
                  etas: $etas,
    eta_change_batches: $eta_change_batches,
  batches_this_segment: $batches_this_segment,
                losses: $loss,
            loss_steps: $loss_step,
           traj_L2_sqs: $traj_L2_sq,
   traj_cos_sq_signeds: $traj_cos_sq_signed,
                    ts: timestamp()}]->
(b:net
          {shorthand: $shorthand,
                 ksv: $ksv,
                loss: $end_loss,
                  ts: timestamp(),
  batches_from_start: $batches_from_start})
"""
    nj.query_write(driver, q, **facts)

In [ ]:
code_strings.append(In[-2])

## Train, recording trajectory

In [ ]:
def train_net_an_increment(net, nps, properties):
    loss = net.losses([training_batch])[0]
    batch_ctr = 0
    while loss > 1e-3:
        batch_ctr_at_seg_start = batch_ctr
        losses = []
        etas = []
        deltas = []
        prior_loss = loss
        while loss / prior_loss > 0.7071 and len(deltas) < 100:
            if not etas or net.eta != etas[-1][1]:
                etas.append([batch_ctr, net.eta])
            loss = net.learn([training_batch])
            if batch_ctr < 100 or batch_ctr % 100 == 0:
                losses.append([batch_ctr, loss])
                deltas.append([batch_ctr, net.deltas()])
            batch_ctr += 1
        #if losses[-1][0] < (batch_ctr-1):
        #    losses.append([batch_ctr, loss])
        if not deltas or deltas[-1][0] < (batch_ctr-1):
            deltas.append((batch_ctr, net.deltas()))
        properties = dict(zip(deltas[0][1]._fields, map(list, (zip(*(v[1] for v in deltas))))))
        #properties = {}
        properties['batch_points'] = [v[0] for v in deltas]
        #properties['etas'] = etas
        properties['etas'], properties['eta_change_batches'] = (list(v) for v in zip(*etas))
        properties['batches_this_segment'] = batch_ctr - batch_ctr_at_seg_start
        properties['ts'] = time.time()
        properties['shorthand'] = net.shorthand
        properties['ksv'] = nps.store(net.state_vector())
        properties['end_loss'] = net.losses([training_batch])[0]
        properties['experiment'] = 'ADC'
        yield properties

In [ ]:
def trainer(net, nps):
    loss = net.losses([training_batch])[0]
    batch_ctr = 0
    while loss > 1e-3:
        batch_ctr_at_seg_start = batch_ctr
        losses = []
        etas = []
        deltas = []
        prior_loss = loss
        while loss / prior_loss > 0.7071 and len(deltas) < 100:
            if not etas or net.eta != etas[-1][1]:
                etas.append([batch_ctr, net.eta])
            loss = net.learn([training_batch])
            if batch_ctr < 100 or batch_ctr % 100 == 0:
                losses.append([batch_ctr, loss])
                deltas.append([batch_ctr, net.deltas()])
            batch_ctr += 1
        #if losses[-1][0] < (batch_ctr-1):
        #    losses.append([batch_ctr, loss])
        if not deltas or deltas[-1][0] < (batch_ctr-1):
            deltas.append((batch_ctr, net.deltas()))
        properties = dict(zip(deltas[0][1]._fields, map(list, (zip(*(v[1] for v in deltas))))))
        #properties = {}
        properties['batch_points'] = [v[0] for v in deltas]
        #properties['etas'] = etas
        properties['etas'], properties['eta_change_batches'] = (list(v) for v in zip(*etas))
        properties['batches_this_segment'] = batch_ctr - batch_ctr_at_seg_start
        properties['ts'] = time.time()
        properties['shorthand'] = net.shorthand
        properties['ksv'] = nps.store(net.state_vector())
        properties['end_loss'] = net.losses([training_batch])[0]
        properties['batches_from_start'] = batch_ctr
        yield properties

In [ ]:
code_strings.append(In[-2])

In [ ]:
def get_starting_facts(net, nps):
    rv = {'shorthand': net.shorthand,
          'ksv': nps.store(net.state_vector()),
          'loss': net.losses([training_batch])[0],
          'head': True,
         }
    return rv

In [ ]:
code_strings.append(In[-2])

## Build the runner

In [ ]:
def create_starting_entry(driver, net, nps, get_starting_facts, q_add_start):
    starting_facts = get_starting_facts(net, nps)
    tj.query_write(driver, q_add_start, **starting_facts)
    return starting_facts['ksv']

In [ ]:
#code_strings.append(In[-2])

In [ ]:
def extend_by_one(driver, net, ksv, nps, observations, add_subsequent):
    with driver.session() as session:
        session.write_transaction(add_subsequent, observations, net)
        print(f"loss {observations['end_loss']}")
        return observations['ksv']

In [ ]:
#code_strings.append(In[-2])

In [ ]:
def run_it(cx, driver, nps):
    net = cx['net']
    add_start = cx['add_net_start_to_graph']
    add_subsequent = cx['add_net_subsequent_to_graph']
    get_starting_facts = cx['get_starting_facts']
    trainer = cx['trainer']

    with driver.session() as session:
        starting_facts = get_starting_facts(net, nps)
        starting_facts['parameters_unikey'] = cx['parameters_unikey']
        add_start(driver, starting_facts)
        prior_ksv = starting_facts['ksv']
        for observations in trainer(net, nps):
            observations['prior_ksv'] = prior_ksv
            prior_ksv = observations['ksv']
            add_subsequent(driver, observations)
            print(f"loss {observations['end_loss']}")

In [ ]:
code_strings.append(In[-2])

* At this point, all the code we need to have in the procedure has been prepared and placed in `code_strings`. It is also in the current notebook context and we can test it here too.
* Some run-specific parameters will be placed in the Parameters nodes created by the methods below.

# Create the Experiment -> Procedure -> Parameters
Create an experiment, add a procedure, add parameters.

## Methods to create/merge

In [ ]:
def create_an_experiment(driver, **kwargs):
    q = """
CREATE (e:Experiment {name: $experiment_name,
                    unikey: $experiment_unikey,
                        ts: timestamp()})
"""
    nj.query_write(driver, q, **kwargs)

In [ ]:
def create_a_procedure(driver, **kwargs):
    q = """
MATCH (e:Experiment {unikey: $experiment_unikey})
CREATE (e)-[:includes]->
(:Procedure {name: $procedure_name,
           unikey: $procedure_unikey,
     code_strings: $code_strings,
               ts: timestamp()})
"""
    nj.query_write(driver, q, **kwargs)

In [ ]:
def create_parameters_to_experiment_procedure(driver, **kwargs):
    q = """
MATCH (proc:Procedure {unikey: $procedure_unikey})
CREATE (proc)-[:incorporates]->
(par:Parameters {unikey: $parameters_unikey,
                   name: $parameters_name,
           code_strings: $code_strings,
                  trial: $trial,
                     ts: timestamp()})
"""
    nj.query_write(driver, q, **kwargs)

## Methods to acquire work

In [ ]:
def get_experiment_key_from_name(driver, **kwargs):
    q="""
MATCH (e:Experiment {name: $name})
RETURN e.unikey as unikey
"""
    records = nj.query_read_return_list(driver, q, **kwargs)
    assert len(records) == 1
    return records[0]['unikey']

In [ ]:
def get_procedure_names_keys_from_experiment_key(driver, **kwargs):
    q = """
MATCH (:Experiment {unikey: $key})
-[:includes]->
(proc:Procedure)
RETURN proc.name as name, proc.unikey as key
"""
    return [(r['name'], r['key']) for r in nj.query_read_yield(driver, q, **kwargs)]

In [ ]:
def get_unstarted_parameters_of_procedure(driver, **kwargs):
    q="""
MATCH (:Procedure {unikey: $procedure_unikey})
-[:incorporates]->
(par:Parameters)
WHERE NOT (par)-[:configures]->(:net)
RETURN par.unikey as unikey
"""
    return [r['unikey'] for r in nj.query_read_yield(driver, q, **kwargs)]

In [ ]:
def get_code_strings_of_experiment_procedure_parameters(driver, **kwargs):
    q = """
MATCH (e:Experiment {unikey: $experiment_unikey})
-[:includes]->
(proc:Procedure {unikey: $procedure_unikey})
-[:incorporates]->
(par:Parameters {unikey: $parameters_unikey})
RETURN proc.code_strings, par.code_strings
"""
    records = nj.query_read_return_list(driver, q, **kwargs)
    assert len(records) == 1
    r = records[0]
    #print(f"r['proc.code_strings'] = {r['proc.code_strings']}")
    #print(f"r['par.code_strings'] = {r['par.code_strings']}")
    return r['proc.code_strings'] + r['par.code_strings']

# Experiment setup

## Switches on what to set up now

In [ ]:
setup_experiment = True
setup_procedure = True
setup_parameters = True

## Configuration

In [ ]:
experiment_name = 't2'
experiment_unikey = secrets.token_urlsafe(16)
procedure_name = 'Train ADCs'
procedure_unikey = secrets.token_urlsafe(16)
parameters = defaultdict(dict)
parameters['go medium']['code_strings'] = ['net.eps = 0.1']
parameters['go medium']['trials'] = 3
parameters['go hard']['code_strings'] = ['net.eps = 1.0']
parameters['go hard']['trials'] = 2


## Do setups according to the switches above

In [ ]:
if setup_experiment:
    create_an_experiment(driver,
        experiment_name=experiment_name,
        experiment_unikey=experiment_unikey,
    )

In [ ]:
if setup_procedure:
    create_a_procedure(driver,
        experiment_unikey=experiment_unikey,
        procedure_name=procedure_name,
        procedure_unikey=procedure_unikey,
        code_strings=code_strings,
    )

In [ ]:
if setup_parameters:
    for name, params in parameters.items():
        for i in range(params['trials']):
            parameters_unikey = secrets.token_urlsafe(16)
            code_strings = [s for s in params['code_strings']]
            code_strings.append(f"parameters_unikey = '{parameters_unikey}'")
            #print(i, code_strings)
            create_parameters_to_experiment_procedure(driver,
                procedure_unikey=procedure_unikey,
                parameters_name=f"{name} {i}",
                parameters_unikey=parameters_unikey,
                code_strings=code_strings,
                trial=i)

---

# Here the experiment is set up in the graph database

We could:
* Run it locally here
* Run it from the code strings we have stored here
* Run it like the bot would, by getting the code from the database
* Launch the bot against it

In [ ]:
run = 'like bot'

In [ ]:
if run == 'local':
    nps = nj.NumpyStore(driver)
    run_it(globals(), driver, nps)

In [ ]:
if run == 'code_strings':
    nps = nj.NumpyStore(driver)
    cx = {}
    for s in code_strings:
        exec(s, cx)
    cx['run_it'](cx, driver, nps)

## Find work and do it
As the bot would

In [ ]:
def get_code_strings_from_db(driver, experiment_name, procedure_name):
    experiment_unikey = get_experiment_key_from_name(driver, name=experiment_name)
    #print(experiment_unikey)
    procedures = dict(get_procedure_names_keys_from_experiment_key(driver, key=experiment_unikey))
    procedure_unikey = procedures[procedure_name]
    #print(procedure_unikey)
    unstarted_parameters = get_unstarted_parameters_of_procedure(driver, procedure_unikey=procedure_unikey)
    parameters_unikey = unstarted_parameters[0]
    #print(parameters_unikey)
    code_strings_from_db = get_code_strings_of_experiment_procedure_parameters(driver,
        experiment_unikey=experiment_unikey,
        procedure_unikey=procedure_unikey,
        parameters_unikey=parameters_unikey)
    return code_strings_from_db

In [ ]:
def now_run_it(driver, code_strings):
    cx = {}
    for s in code_strings:
        exec(s, cx)
    nps = nj.NumpyStore(driver)
    cx['run_it'](cx, driver, nps)

In [ ]:
if run == 'like bot':
    code_strings = get_code_strings_from_db(driver, 't2', 'Train ADCs')
    now_run_it(driver, code_strings)

In [ ]:
if run == 'bot':
    raise NotImplementedError

___

# Stop

In [ ]:
assert False, "stop here"

In [ ]:
if True:
    experiment_unikey = get_experiment_key_from_name(driver, name=experiment_name)
    print(experiment_unikey)
    procedures = dict(get_procedure_names_keys_from_experiment_key(driver, key=experiment_unikey))
    procedure_unikey = procedures[procedure_name]
    unstarted_parameters = get_unstarted_parameters_of_procedure(driver, procedure_unikey=procedure_unikey)
    parameters_unikey = unstarted_parameters[0]


In [ ]:
code_strings_from_db = get_code_strings_from_db(driver, 't2', 'Train ADCs')

In [ ]:
print(('\n\n/' + '*'*80 + '/\n').join(code_strings_from_db))

In [ ]:
get_code_strings_from_db(driver, 't2', 'Train ADCs')

In [ ]:
print(('\n\n/' + '*'*80 + '/\n').join(code_strings))